In [ ]:
import json
import os
import sys
from urllib.request import Request, urlopen

import pandas as pd
import requests
from bs4 import BeautifulSoup
from dotenv import load_dotenv  # pip install python-dotenv
from neo4j import GraphDatabase
from SPARQLWrapper import JSON, SPARQLWrapper
from tqdm.notebook import tqdm

pd.set_option("max_colwidth", 400)
pd.set_option("display.max_rows", None)

# make sure a .env file exists in the same directory, with a line like this:
# KG_PWD=<insert password here>
load_dotenv()

In [ ]:
os.chdir("../../")

In [ ]:
KG_PWD = os.environ.get("KG_PWD")

In [ ]:
class Neo4jConnection:
    def __init__(self, uri, user, pwd):
        self.__uri = uri
        self.__user = user
        self.__pwd = pwd
        self.__driver = None
        try:
            self.__driver = GraphDatabase.driver(
                self.__uri, auth=(self.__user, self.__pwd)
            )
        except Exception as e:
            print("Failed to create the driver:", e)

    def close(self):
        if self.__driver is not None:
            self.__driver.close()

    def query(self, query, db=None):
        assert self.__driver is not None, "Driver not initialized!"
        session = None
        response = None
        try:
            session = (
                self.__driver.session(database=db)
                if db is not None
                else self.__driver.session()
            )
            response = list(session.run(query))
        except Exception as e:
            print("Query failed:", e)
        finally:
            if session is not None:
                session.close()
        return response


def get_results(query, endpoint_url="https://query.wikidata.org/sparql"):
    """
    For querying wikidata
    """
    user_agent = "WDQS-example Python/%s.%s" % (
        sys.version_info[0],
        sys.version_info[1],
    )
    # TODO adjust user agent; see https://w.wiki/CX6
    sparql = SPARQLWrapper(endpoint_url, agent=user_agent)
    sparql.setQuery(query)
    sparql.setReturnFormat(JSON)
    return sparql.query().convert()

We're going to make a big list of dictionaries and turn that into a JSON lines file.

In [ ]:
json_file = []

## FORM

In [ ]:
KG_PWD = "nottobeshared"

In [ ]:
conn = Neo4jConnection(
    uri="bolt+s://knowledge-graph.integration.govuk.digital:7687",
    user="neo4j",
    pwd=KG_PWD,
)

q = """
MATCH (n)
WHERE n.documentType='form'
RETURN n.title, n.description, n.text
"""
r = conn.query(q)

r = pd.DataFrame(r, columns=r[0]._Record__keys)

In [ ]:
r.head()

In [ ]:
form_target_df = pd.DataFrame()

In [ ]:
field = "n.text"

subset = r[[field]]
subset = subset.rename(columns={field: "text"})
subset["meta"] = field

form_target_df = form_target_df.append(subset)

In [ ]:
form_target_df.shape

In [ ]:
form_target_df.sample(10)

In [ ]:
def sentences_to_jsonl(dataframe, sentence_col, meta_cols, outfile):
    dict_lines = []
    for i, row in tqdm(dataframe.iterrows()):
        # dict_line = {"text": sentence, "meta": {"base_path": base_path, "content_id": c_id}}
        dict_line = {"text": row["text"], "meta": {i: row[i] for i in meta_cols}}
        dict_lines.append(dict_line)
    with open(outfile, "w") as jsonlfile:
        jsonlfile.write("\n".join(json.dumps(j) for j in dict_lines))

In [ ]:
sentences_to_jsonl(
    dataframe=form_target_df,
    sentence_col="text",
    meta_cols=["meta"],
    outfile="data/interim/targeted_extraction_form.jsonl",
)

In [ ]:
def has_numbers(inputString):
    return any(char.isdigit() for char in inputString)


def has_alpha(inputString):
    return any(char.isalpha() for char in inputString)


def has_special(inputString):
    return any(not char.isalnum() for char in inputString)


def extractFormName(inputString):
    inputString = inputString.replace(":", "")
    outputString = [
        token
        for token in inputString.split()
        if has_numbers(token) and has_alpha(token) and not has_special(token)
    ]
    if outputString:
        return outputString[0]
    else:
        return ""

In [ ]:
forms = [
    {"label": "FORM", "pattern": pattern}
    for pattern in [form for form in r["n.title"].apply(extractFormName) if form]
]

In [ ]:
json_file.extend(forms)

### Extracting sentences containing forms

In [ ]:
conn = Neo4jConnection(
    uri="bolt+s://knowledge-graph.integration.govuk.digital:7687",
    user="neo4j",
    pwd=KG_PWD,
)

forms_list = [x["pattern"] for x in forms]

q = """
WITH {items} AS items
MATCH (n)
// This bit splits up a text into a list of tokens, then determines if any forms
// are in that list
WHERE any(item IN items WHERE item IN split(n.text, ' '))
RETURN n.name, n.text
LIMIT 500
""".format(
    items=forms_list
)

r = conn.query(q)

p = pd.DataFrame(r, columns=r[0]._Record__keys)

In [ ]:
form_examples = []

# iterate over each page and look at its text
for text in p["n.text"]:
    # extract the sentences from the page with a crude heuristic, then iterate over those
    for sentence in text.split(". "):
        # extract the tokens from the sentence
        # this avoids partial matches
        split_sentence = sentence.split()
        # look at each form and check if its in the sentence
        for form in forms_list:
            # add example of form in use
            if form in split_sentence:
                form_examples.append({"form": form, "sentence": sentence})

In [ ]:
form_examples = pd.DataFrame(form_examples)
form_examples.head()

In [ ]:
form_examples = form_examples.drop_duplicates(subset=["sentence"])
form_examples.shape

In [ ]:
form_examples = form_examples.sample(1000)

Get into prodigy format

In [ ]:
def sentences_to_jsonl(dataframe, sentence_col, outfile):
    """
    Convert a column containing list of sentences for each row into .jsonl file format for Prodigy. Specify metacols and file path to save .jsonl to.
    """
    dict_lines = []
    for i, row in tqdm(dataframe.head().iterrows()):
        for sentence in row[sentence_col]:
            # dict_line = {"text": sentence, "meta": {"base_path": base_path, "content_id": c_id}}
            dict_line = {"text": sentence, "meta": {"source": "TE Notebook"}}
            dict_lines.append(dict_line)
    with open(outfile, "w") as jsonlfile:
        jsonlfile.write("\n".join(json.dumps(j) for j in dict_lines))

In [ ]:
def sentences_to_jsonl(dataframe, sentence_col, outfile):
    """
    Convert a column containing list of sentences for each row into .jsonl file format for Prodigy. Specify metacols and file path to save .jsonl to.
    """
    dict_lines = []
    for sentence in dataframe[sentence_col]:
        # dict_line = {"text": sentence, "meta": {"base_path": base_path, "content_id": c_id}}
        dict_line = {"text": sentence, "meta": {"source": "TE Notebook"}}
        dict_lines.append(dict_line)
    with open(outfile, "w") as jsonlfile:
        jsonlfile.write("\n".join(json.dumps(j) for j in dict_lines))

In [ ]:
sentences_to_jsonl(
    form_examples,
    "sentence",
    "/Users/roryhurley/Documents/GitHub/govuk-content-metadata/data/interim/targeted_extraction_forms_100_falsepositives.jsonl",
)

## PERSON NAME

### govGraph names

In [ ]:
conn = Neo4jConnection(
    uri="bolt+s://knowledge-graph.integration.govuk.digital:7687",
    user="neo4j",
    pwd=KG_PWD,
)

q = """
MATCH (n)
WHERE n.documentType='person'
RETURN n.title
"""
r = conn.query(q)

person_name = [
    {"label": "PERSON (NAME)", "pattern": pattern} for pattern in pd.DataFrame(r)[0]
]

In [ ]:
json_file.extend(person_name)

### wikiData names

In [ ]:
# get 10000 examples of everything that is an 'instance of' 'human'

query = """SELECT DISTINCT ?item ?itemLabel WHERE {
  SERVICE wikibase:label { bd:serviceParam wikibase:language "en". }
  {
    SELECT DISTINCT ?item WHERE {
      ?item p:P31 ?statement0.
      ?statement0 (ps:P31/(wdt:P279*)) wd:Q5.
    }
    LIMIT 10000
  }
}"""

names = get_results(query)
names = pd.json_normalize(names["results"]["bindings"])["itemLabel.value"]

In [ ]:
# remove Q numbers and very long names

person_names_wiki = [
    {"label": "PERSON (NAME)", "pattern": pattern}
    for pattern in [
        name for name in names if not name[-1].isdigit() and len(name.split()) < 4
    ]
]

In [ ]:
json_file.extend(person_names_wiki)

## JOBS

In [ ]:
# get everything that is an 'instance of' 'job'

query = """SELECT DISTINCT ?item ?itemLabel WHERE {
  SERVICE wikibase:label { bd:serviceParam wikibase:language "en". }
  {
    SELECT DISTINCT ?item WHERE {
      ?item p:P31 ?statement0.
      ?statement0 (ps:P31/(wdt:P279*)) wd:Q192581.
    }
  }
}"""

jobs = get_results(query)
jobs = pd.json_normalize(jobs["results"]["bindings"])["itemLabel.value"]

In [ ]:
# remove Q numbers and very long job titles
jobs = [job for job in jobs if not job[-1].isdigit() and len(job.split()) < 4]

In [ ]:
# get everything that is an 'instance of' 'profession'

query = """SELECT DISTINCT ?item ?itemLabel WHERE {
  SERVICE wikibase:label { bd:serviceParam wikibase:language "en". }
  {
    SELECT DISTINCT ?item WHERE {
      ?item p:P31 ?statement0.
      ?statement0 (ps:P31/(wdt:P279*)) wd:Q28640.
    }
  }
}"""

professions = get_results(query)
professions = pd.json_normalize(professions["results"]["bindings"])["itemLabel.value"]

In [ ]:
# remove Q numbers and very long job titles
professions = [
    profession
    for profession in professions
    if not profession[-1].isdigit() and len(profession.split()) < 4
]

In [ ]:
# get all the unique jobs and professions
unique_jobs = [
    {"label": "PERSON (PROFESSION)", "pattern": pattern}
    for pattern in list(set(jobs) | set(professions))
]

In [ ]:
json_file.extend(unique_jobs)

## PERSON ROLE

#### Ministerial roles

In [ ]:
conn = Neo4jConnection(
    uri="bolt+s://knowledge-graph.integration.govuk.digital:7687",
    user="neo4j",
    pwd=KG_PWD,
)

q = """
MATCH (n)
WHERE n.documentType='ministerial_role'
RETURN n.title
"""
r = conn.query(q)

ministerial_roles = [
    {"label": "PERSON (ROLE)", "pattern": pattern} for pattern in pd.DataFrame(r)[0]
]

In [ ]:
json_file.extend(ministerial_roles)

## ORGANISATION

In [ ]:
conn = Neo4jConnection(
    uri="bolt+s://knowledge-graph.integration.govuk.digital:7687",
    user="neo4j",
    pwd=KG_PWD,
)

q = """
MATCH (n)
WHERE n.documentType='organisation'
RETURN n.name, n.title
"""
r = conn.query(q)

public_organisations = pd.DataFrame(r, columns=r[0]._Record__keys)

In [ ]:
org_acronyms = []

for (_, row) in tqdm(public_organisations.iterrows()):
    try:
        pass
        response = requests.get(
            f"https://www.gov.uk/api/search.json?filter_link=" + row["n.name"]
        )
        org_acronyms.append(
            response.json()["results"][0]["organisations"][0]["acronym"]
        )
    except:
        continue

In [ ]:
public_organisations = [
    {"label": "ORGANISATION (PUBLIC)", "pattern": pattern}
    for pattern in public_organisations["n.title"]
]

org_acronyms = [
    {"label": "ORGANISATION (PUBLIC)", "pattern": pattern} for pattern in org_acronyms
]

In [ ]:
json_file.extend(public_organisations)
json_file.extend(org_acronyms)

### Departments of the UK Government

In [ ]:
# get everything than is an 'instance of' 'department of the United Kingdom Government'

query = """SELECT DISTINCT ?item ?itemLabel WHERE {
  SERVICE wikibase:label { bd:serviceParam wikibase:language "en". }
  {
    SELECT DISTINCT ?item WHERE {
      ?item p:P31 ?statement0.
      ?statement0 (ps:P31/(wdt:P279*)) wd:Q2500378.
    }
  }
}"""

gov_dept = get_results(query)
gov_dept = pd.json_normalize(gov_dept["results"]["bindings"])["itemLabel.value"]

In [ ]:
gov_dept = [
    {"label": "ORGANISATION (PUBLIC)", "pattern": pattern} for pattern in gov_dept
]

In [ ]:
json_file.extend(gov_dept)

### Nasdaq-100 companies

In [ ]:
# get everything that is 'part of' 'Nasdaq-100'

query = """SELECT DISTINCT ?item ?itemLabel WHERE {
  SERVICE wikibase:label { bd:serviceParam wikibase:language "en". }
  {
    SELECT DISTINCT ?item WHERE {
      ?item p:P361 ?statement0.
      ?statement0 (ps:P361/(wdt:P279*)) wd:Q507306.
    }
  }
}"""

nasdaq_100 = get_results(query)
nasdaq_100 = pd.json_normalize(nasdaq_100["results"]["bindings"])["itemLabel.value"]

In [ ]:
nasdaq_100 = [
    {"label": "ORGANISATION (PRIVATE)", "pattern": pattern} for pattern in nasdaq_100
]

In [ ]:
json_file.extend(nasdaq_100)

### FTSE-100 companies

In [ ]:
# get everything that is 'part of' 'FTSE 100 Index'

query = """SELECT DISTINCT ?item ?itemLabel WHERE {
  SERVICE wikibase:label { bd:serviceParam wikibase:language "en". }
  {
    SELECT DISTINCT ?item WHERE {
      ?item p:P361 ?statement0.
      ?statement0 (ps:P361/(wdt:P279*)) wd:Q466496.
    }
  }
}"""

ftse_100 = get_results(query)
ftse_100 = pd.json_normalize(ftse_100["results"]["bindings"])["itemLabel.value"]

In [ ]:
ftse_100 = [
    {"label": "ORGANISATION (PRIVATE)", "pattern": pattern} for pattern in ftse_100
]

In [ ]:
json_file.extend(ftse_100)

## EVENTS

### Life event

In [ ]:
site = "https://simplicable.com/en/life-events"
hdr = {"User-Agent": "Mozilla/5.0"}
req = Request(site, headers=hdr)
page = urlopen(req)
soup = BeautifulSoup(page)
life_events = [
    td.find("span", class_="blogy").text
    for td in soup.findAll("td", class_="tdFlatList")
]

In [ ]:
life_events = [
    {"label": "EVENT (LIFE EVENT)", "pattern": pattern} for pattern in life_events
]

In [ ]:
json_file.extend(life_events)

## SCHEME

In [ ]:
schemes = []
for page_num in range(1, 5):
    page = requests.get(f"https://www.gov.uk/business-finance-support?page={page_num}")
    soup = BeautifulSoup(page.content, "html.parser")

    for scheme in soup.find_all("a", class_="gem-c-document-list__item-title"):
        schemes.append(scheme.text)

schemes = [scheme.split("-")[0].strip() for scheme in schemes]

In [ ]:
schemes = [{"label": "SCHEME", "pattern": pattern} for pattern in schemes]

In [ ]:
json_file.extend(schemes)

## LOCATIONS

### Countries and capitals

In [ ]:
# get countries and capitals

query = """#List of present-day countries and capital(s)
SELECT DISTINCT ?country ?countryLabel ?capital ?capitalLabel
WHERE
{
  ?country wdt:P31 wd:Q3624078 .
  #not a former country
  FILTER NOT EXISTS {?country wdt:P31 wd:Q3024240}
  #and no an ancient civilisation (needed to exclude ancient Egypt)
  FILTER NOT EXISTS {?country wdt:P31 wd:Q28171280}
  OPTIONAL { ?country wdt:P36 ?capital } .

  SERVICE wikibase:label { bd:serviceParam wikibase:language "en" }
}
ORDER BY ?countryLabel"""

countries_capitals = get_results(query)
countries_capitals = pd.json_normalize(countries_capitals["results"]["bindings"])[
    ["countryLabel.value", "capitalLabel.value"]
]

In [ ]:
countries = [
    {"label": "LOCATION (GPE)", "pattern": pattern}
    for pattern in countries_capitals["countryLabel.value"]
]
capitals = [
    {"label": "LOCATION (GPE)", "pattern": pattern}
    for pattern in countries_capitals["capitalLabel.value"]
]

In [ ]:
json_file.extend(countries)
json_file.extend(capitals)

### United Kingdom Counties

In [ ]:
# get everything with 'country' 'United Kingdom' AND 'instance of' 'county'

query = """SELECT DISTINCT ?item ?itemLabel WHERE {
  SERVICE wikibase:label { bd:serviceParam wikibase:language "en". }
  {
    SELECT DISTINCT ?item WHERE {
      ?item p:P17 ?statement0.
      ?statement0 (ps:P17/(wdt:P279*)) wd:Q145.
      ?item p:P31 ?statement1.
      ?statement1 (ps:P31/(wdt:P279*)) wd:Q28575.
    }
  }
}"""

counties = get_results(query)
counties = pd.json_normalize(counties["results"]["bindings"])["itemLabel.value"]

In [ ]:
counties = [{"label": "GPE", "pattern": pattern} for pattern in counties]

In [ ]:
json_file.extend(counties)

###  Split multi word entities

In [ ]:
split_json_file = [
    {
        "label": file["label"],
        "pattern": [{"lower": word.lower()} for word in str(file["pattern"]).split()],
    }
    for file in json_file
]

## Write to JSON lines

In [ ]:
with open("patterns.jsonl", "w") as outfile:
    for entry in split_json_file:
        json.dump(entry, outfile)
        outfile.write("\n")